In [1]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'commodities-s4' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [ ]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_4462b5d_188b33f6f40') # Job ID inserted based on the query results selected to explore
print(job.query)

In [3]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_4462b5d_188b33f6f40') # Job ID inserted based on the query results selected to explore
results = job.to_dataframe()
names=set(results["DSMnemonic"])
names=list(names)

In [11]:
import random
import pandas as pd

def select(names,num,results):
  l=[]
  for i in range(num):
    l.append(random.randint(0,len(names)))
  nam=[names[i] for i in l]
  datas=[]
  start=int(random.randint(0,9)/10*1500)
  end=start+random.randint(300,3000-start)
  print("start=%s,end=%s"%(start,end))
  datamain=results.query("DSMnemonic=='%s'"%nam[0])
  datamain=datamain[["Date_","Close_"]]
  datamain.drop_duplicates(subset=["Date_"], keep='first', inplace=True)
  datamain=datamain.sort_values(by="Date_")
  datamain["Date_"]=pd.to_datetime(datamain["Date_"])
  datamain["Close_"] = pd.to_numeric(datamain["Close_"])
  datamain=datamain[start:end]
  datamain=datamain.rename(columns={"Date_": "ds", "Close_": "%s"%nam[0]})

  for name in nam:
    if name ==nam[0]:
      continue
    data=results.query("DSMnemonic=='%s'"%name)
    data=data[["Date_","Close_"]]
    data.drop_duplicates(subset=["Date_"], keep='first', inplace=True)
    data=data.sort_values(by="Date_")
    data["Date_"]=pd.to_datetime(data["Date_"])
    data["Close_"] = pd.to_numeric(data["Close_"])
    data=data[start:end]
    data=data.rename(columns={"Date_": "ds", "Close_": "%s"%name})
    datamain=pd.merge(datamain,data,on="ds", how="outer")
  return datamain

In [12]:
from google.colab import files
df=select(names,200,results)
df.to_csv('data_3.csv')
files.download('data_3.csv')

start=600,end=2770


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>